## Second experiment - second configuration.

In [4]:
import csv
import os
import glob
import random
import numpy as np
from collections import OrderedDict

from analysis_tools import load_raw

import mne
from mne import Epochs, find_events
from mne.decoding import Vectorizer

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report

from pyriemann.estimation import ERPCovariances
from pyriemann.tangentspace import TangentSpace
from pyriemann.classification import MDM
from pyriemann.spatialfilters import Xdawn

### The generated epochs and authentication datasets will be saved in the "epochs" directory. If the directory does not exist, it is created.

In [5]:
path = 'second_experiment/second_configuration/epochs'

if not os.path.exists(path):
    os.makedirs(path)

### Application of Notch to attenuate the frequency at 50 Hz, the sixth-order Butterworth band-pass filter with cut-off frequencies of 1-17 Hz, and ICA. After their application, the framework generates the epochs.

In [ ]:
def process_by_subject(subject_name):
    count = 1
    datasets = sorted(glob.glob('data/'+ subject_name + '_*.csv'))
    array_epochs = []
    for dataset in datasets:
        sampling_rate = 256

        ch_names = {}
        
        raw = load_raw(dataset, sfreq=sampling_rate, stim_ind=8, replace_ch_names=None, ch_ind=[0, 1, 2, 3, 4, 5, 6, 7])
        
        for i, chn in enumerate(raw.ch_names):
            ch_names[chn] = i

        raw_notch = raw.copy().notch_filter([50.0])

        iir_params = dict(order=6, ftype='butter')
        raw_notch_and_filter = raw_notch.copy().filter(1, 17, method='iir', iir_params=iir_params)
            
        ica = mne.preprocessing.ICA(n_components=8, random_state=97)
        ica.fit(raw_notch_and_filter)
        
        raw_notch_and_filter_ica = raw_notch_and_filter.copy()
        
        ica.exclude = []
        eog_inds, eog_scores = ica.find_bads_eog(raw_notch_and_filter_ica, ['Fp1','Fp2'], threshold=1.5)
        ica.exclude = eog_inds
                
        ica.apply(raw_notch_and_filter_ica)

        events = find_events(raw_notch_and_filter_ica, shortest_event=1) 
                
        event_id = {'Target': 1, 'NoTarget': 2}
        reject = {'eeg': 100e-6}

        epochs = Epochs(raw_notch_and_filter_ica, events=events, event_id=event_id, tmin=-0.1, tmax=0.8, reject=reject, preload=True)
        epochs.pick_types(eeg=True)
    
        array_epochs.append(epochs)
        
        count = count + 1
    
    return mne.concatenate_epochs(array_epochs, add_offset=True)

process_by_subject("user_01").save('second_experiment/second_configuration/epochs/extracted_epochs_user_01_second_experiment_second_configuration-epo.fif', overwrite = True)
process_by_subject("user_02").save('second_experiment/second_configuration/epochs/extracted_epochs_user_02_second_experiment_second_configuration-epo.fif', overwrite = True)
process_by_subject("user_03").save('second_experiment/second_configuration/epochs/extracted_epochs_user_03_second_experiment_second_configuration-epo.fif', overwrite = True)
process_by_subject("user_04").save('second_experiment/second_configuration/epochs/extracted_epochs_user_04_second_experiment_second_configuration-epo.fif', overwrite = True)
process_by_subject("user_05").save('second_experiment/second_configuration/epochs/extracted_epochs_user_05_second_experiment_second_configuration-epo.fif', overwrite = True)
process_by_subject("user_06").save('second_experiment/second_configuration/epochs/extracted_epochs_user_06_second_experiment_second_configuration-epo.fif', overwrite = True)
process_by_subject("user_07").save('second_experiment/second_configuration/epochs/extracted_epochs_user_07_second_experiment_second_configuration-epo.fif', overwrite = True)
process_by_subject("user_08").save('second_experiment/second_configuration/epochs/extracted_epochs_user_08_second_experiment_second_configuration-epo.fif', overwrite = True)
process_by_subject("user_09").save('second_experiment/second_configuration/epochs/extracted_epochs_user_09_second_experiment_second_configuration-epo.fif', overwrite = True)
process_by_subject("user_10").save('second_experiment/second_configuration/epochs/extracted_epochs_user_10_second_experiment_second_configuration-epo.fif', overwrite = True)

### Generation of five authentication datasets for each subject.

In [ ]:
def get_authentication_epochs(subject_name):    
    epochs_user_01 = mne.read_epochs('second_experiment/second_configuration/epochs/extracted_epochs_user_01_second_experiment_second_configuration-epo.fif', preload=False)
    epochs_user_02 = mne.read_epochs('second_experiment/second_configuration/epochs/extracted_epochs_user_02_second_experiment_second_configuration-epo.fif', preload=False)
    epochs_user_03 = mne.read_epochs('second_experiment/second_configuration/epochs/extracted_epochs_user_03_second_experiment_second_configuration-epo.fif', preload=False)
    epochs_user_04 = mne.read_epochs('second_experiment/second_configuration/epochs/extracted_epochs_user_04_second_experiment_second_configuration-epo.fif', preload=False)
    epochs_user_05 = mne.read_epochs('second_experiment/second_configuration/epochs/extracted_epochs_user_05_second_experiment_second_configuration-epo.fif', preload=False)
    epochs_user_06 = mne.read_epochs('second_experiment/second_configuration/epochs/extracted_epochs_user_06_second_experiment_second_configuration-epo.fif', preload=False)
    epochs_user_07 = mne.read_epochs('second_experiment/second_configuration/epochs/extracted_epochs_user_07_second_experiment_second_configuration-epo.fif', preload=False)
    epochs_user_08 = mne.read_epochs('second_experiment/second_configuration/epochs/extracted_epochs_user_08_second_experiment_second_configuration-epo.fif', preload=False)
    epochs_user_09 = mne.read_epochs('second_experiment/second_configuration/epochs/extracted_epochs_user_09_second_experiment_second_configuration-epo.fif', preload=False)
    epochs_user_10 = mne.read_epochs('second_experiment/second_configuration/epochs/extracted_epochs_user_10_second_experiment_second_configuration-epo.fif', preload=False)
    
    subjects = ["user_01", "user_02", "user_03", "user_04", "user_05", "user_06", "user_07", "user_08", "user_09", "user_10"]
    epochs_subjects = [epochs_user_01, epochs_user_02, epochs_user_03, epochs_user_04, epochs_user_05, epochs_user_06, epochs_user_07, epochs_user_08, epochs_user_09, epochs_user_10]
    
    epochs_train = []
    epochs_test = []
    
    index_subject = subjects.index(subject_name)
    subject = 0
    for epochs in epochs_subjects:
        no_targets = np.count_nonzero(epochs.events[:, -1]==2)
        
        index_no_targets = []
        y = epochs.events[:, -1]

        while(no_targets != 0):
            position = random.randint(0, len(y)-1)
            if y[position] == 2 and position not in index_no_targets:
                index_no_targets.append(position)
                no_targets -= 1

        epochs.drop(index_no_targets)
        
        if subject == index_subject:
            epochs_subject = epochs
    
        subject += 1
        
    del epochs_subjects[index_subject]

    targets_subject = np.count_nonzero(epochs_subject.events[:, -1]==1)
    number_rest_subjects = len(epochs_subjects)
    targets_rest_subjects = targets_subject // number_rest_subjects
    targets_last_subject = (targets_subject - (targets_rest_subjects * number_rest_subjects)) + targets_rest_subjects
    
    num_epochs_train = int(targets_subject * 0.7)
    index_epochs_subject = np.random.permutation(targets_subject)
    index_train_subect = index_epochs_subject[:num_epochs_train]
    index_test_subject = index_epochs_subject[num_epochs_train:]
    
    epochs_train_subject = epochs_subject[index_train_subect]
    epochs_test_subject = epochs_subject[index_test_subject]
    
    epochs_train.append(epochs_train_subject)
    epochs_test.append(epochs_test_subject)
    
    number_attackers = 3
    
    attacker_index = random.sample(range(number_rest_subjects), number_attackers)
    
    subject = 0
    for i, epochs in enumerate(epochs_subjects):
        targets = np.count_nonzero(epochs.events[:, -1]==1)
        index_leftover_targets = []
        y = epochs.events[:, -1]

        if(subject == number_rest_subjects - 1):
            targets_out = targets - targets_last_subject

        else:
            targets_out = targets - targets_rest_subjects

        while(targets_out != 0):
            position = random.randint(0, len(y)-1)
            if position not in index_leftover_targets:
                index_leftover_targets.append(position)
                targets_out -= 1

        epochs.drop(index_leftover_targets)

        for j in range (len(epochs.events)):
            epochs.events[j][2] = 2
              
        subject += 1
        if i in attacker_index:
            epochs_test.append(epochs)
        else:
            epochs_train.append(epochs)
            
    return mne.concatenate_epochs(epochs_train, add_offset=True), mne.concatenate_epochs(epochs_test, add_offset=True)

subjects = ["user_01", "user_02", "user_03", "user_04", "user_05", "user_06", "user_07", "user_08", "user_09", "user_10"]

for subject in subjects:
    for i in range(5):
        epochs_train, epochs_test = get_authentication_epochs(subject)
        epochs_train.save('second_experiment/second_configuration/epochs/authentication_epochs_train_{}_{}_second_experiment_second_configuration-epo.fif'.format(subject, i), overwrite = True)
        epochs_test.save('second_experiment/second_configuration/epochs/authentication_epochs_test_{}_{}_second_experiment_second_configuration-epo.fif'.format(subject, i), overwrite = True)

### The results obtained will be saved in the "results" directory. If the directory does not exist, it is created.

In [8]:
path = 'second_experiment/second_configuration/results'

if not os.path.exists(path):
    os.makedirs(path)

### Generation of a CSV file that will contain the results obtained in the authentication process.

In [9]:
header = ['Option/Classifier', 'Classifier1-F1Score', 'Classifier1-EER', 'Classifier1-FAR', 'Classifier1-FRR', 'Classifier2-F1Score', 'Classifier2-EER', 'Classifier2-FAR', 'Classifier2-FRR', 'Classifier3-F1Score', 'Classifier3-EER', 'Classifier3-FAR', 'Classifier3-FRR', 'Classifier4-F1Score', 'Classifier4-EER', 'Classifier4-FAR', 'Classifier4-FRR', 'Classifier5-F1Score', 'Classifier5-EER', 'Classifier5-FAR', 'Classifier5-FRR', 'Classifier6-F1Score', 'Classifier6-EER', 'Classifier6-FAR', 'Classifier6-FRR', 'Classifier7-F1Score', 'Classifier7-EER', 'Classifier7-FAR', 'Classifier7-FRR', 'Classifier8-F1Score', 'Classifier8-EER', 'Classifier8-FAR', 'Classifier8-FRR', 'Classifier9-F1Score', 'Classifier9-EER', 'Classifier9-FAR', 'Classifier9-FRR', 'Classifier10-F1Score', 'Classifier10-EER', 'Classifier10-FAR', 'Classifier10-FRR', 'Classifier11-F1Score', 'Classifier11-EER', 'Classifier11-FAR', 'Classifier11-FRR', 'Classifier12-F1Score', 'Classifier12-EER', 'Classifier12-FAR', 'Classifier12-FRR', 'Classifier13-F1Score', 'Classifier13-EER', 'Classifier13-FAR', 'Classifier13-FRR', 'Classifier14-F1Score', 'Classifier14-EER', 'Classifier14-FAR', 'Classifier14-FRR', 'Classifier15-F1Score', 'Classifier15-EER', 'Classifier15-FAR', 'Classifier15-FRR', 'Classifier16-F1Score', 'Classifier16-EER', 'Classifier16-FAR', 'Classifier16-FRR', 'Classifier17-F1Score', 'Classifier17-EER', 'Classifier17-FAR', 'Classifier17-FRR']
with open('second_experiment/second_configuration/results/results_second_experiment_second_configuration.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    
    writer.writerow(header)

### Authentication process using binary classification.

In [ ]:
clfs = OrderedDict()

clfs['Clasificador I'] = make_pipeline(Vectorizer(), StandardScaler(), LogisticRegression())
clfs['Clasificador II'] = make_pipeline(Vectorizer(), LDA(shrinkage='auto', solver='eigen'))
clfs['Clasificador III'] = make_pipeline(Xdawn(2, classes=[1]), Vectorizer(), LDA(shrinkage='auto', solver='eigen'))
clfs['Clasificador IV'] = make_pipeline(ERPCovariances(estimator='oas'), TangentSpace(), LogisticRegression())
clfs['Clasificador V'] = make_pipeline(ERPCovariances(estimator='oas'), MDM())
clfs['Clasificador VI'] = make_pipeline(Vectorizer(), RandomForestClassifier(random_state=42))
clfs['Clasificador VII'] = make_pipeline(Vectorizer(), QDA())
clfs['Clasificador VIII'] = make_pipeline(Vectorizer(), KNeighborsClassifier(n_neighbors=50))
clfs['Clasificador IX'] = make_pipeline(Xdawn(2, classes=[1]), Vectorizer(), RandomForestClassifier(random_state=42))
clfs['Clasificador X'] = make_pipeline(ERPCovariances(estimator='oas'), TangentSpace(), RandomForestClassifier(random_state=42))
clfs['Clasificador XI'] = make_pipeline(ERPCovariances(estimator='oas'), Vectorizer(), RandomForestClassifier(random_state=42))
clfs['Clasificador XII'] = make_pipeline(Xdawn(2, classes=[1]), Vectorizer(), QDA())
clfs['Clasificador XIII'] = make_pipeline(ERPCovariances(estimator='oas'), TangentSpace(), QDA())
clfs['Clasificador XIV'] = make_pipeline(ERPCovariances(estimator='oas'), Vectorizer(), QDA())
clfs['Clasificador XV'] = make_pipeline(Xdawn(2, classes=[1]), Vectorizer(), KNeighborsClassifier(n_neighbors=50))
clfs['Clasificador XVI'] = make_pipeline(ERPCovariances(estimator='oas'), TangentSpace(), KNeighborsClassifier(n_neighbors=50))
clfs['Clasificador XVII'] = make_pipeline(ERPCovariances(estimator='oas'), Vectorizer(), KNeighborsClassifier(n_neighbors=50))

def calculate_eer_far_frr(tp, fp, tn, fn):
    if (tp + fp) > 0:
        far = fp / (tn + fp)
    else:
        far = 0.0
    
    if (tp + fn) > 0:
        frr = fn / (tp + fn)
    else:
        frr = 0.0
    
    eer = (fp + fn) / (tp + tn + fp + fn)
    
    return eer, far, frr

def authentication_by_subject(subject_name, epochs_train, epochs_test, experiment):
   
    option = 'Second_experiment_second_configuration_' + subject_name + '_' + str(experiment)
    
    data = []
    data.append(option)
    
    X_train = epochs_train.get_data() * 1e6
    times_train = epochs_train.times
    y_train = epochs_train.events[:, -1]
    
    X_test = epochs_test.get_data() * 1e6
    times_test = epochs_test.times
    y_test = epochs_test.events[:, -1]
        
    for m in clfs:
        clfs[m].fit(X_train, y_train)
        y_pred = clfs[m].predict(X_test)
        report = classification_report(y_test, y_pred, output_dict=True)
        f1_score = report['weighted avg']['f1-score']

        cm = confusion_matrix(y_test, y_pred)

        tn, fp, fn, tp = cm.ravel()

        eer, far, frr = calculate_eer_far_frr(tp, fp, tn, fn)

        data.append(round(f1_score, 2))
        data.append(round(eer, 2))
        data.append(round(far, 2))
        data.append(round(frr, 2))
            
        
    with open('second_experiment/second_configuration/results/results_second_experiment_second_configuration.csv', 'a') as f:
        writer = csv.writer(f)
    
        writer.writerow(data)
        
        f.close()
              
subjects = ["user_01", "user_02", "user_03", "user_04", "user_05", "user_06", "user_07", "user_08", "user_09", "user_10"]

for subject in subjects:
    for i in range(5):
        authentication_by_subject(subject, mne.read_epochs('second_experiment/second_configuration/epochs/authentication_epochs_train_{}_{}_second_experiment_second_configuration-epo.fif'.format(subject, i), preload=False), mne.read_epochs('second_experiment/second_configuration/epochs/authentication_epochs_test_{}_{}_second_experiment_second_configuration-epo.fif'.format(subject, i), preload=False), i)